In [3]:
import cv2
import pytesseract
import re
import numpy as np
import pandas as pd
import joblib
import easyocr

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [20]:
def extract_img_1(path, keywords):
    reader = easyocr.Reader(['en'],gpu=False) # this needs to run only once to load the model into memory
    img = path
    output = reader.readtext(img,detail=1)
    dictionary = {}
    for i in output:
               # print(i)
                probability = i[-1]
                if probability >= 0.6:
                    text = i[1] 
                for keyword in keywords :
                    try :
                        try :
                            if re.search(keyword, text, re.IGNORECASE):                     
                                if '=' in text :
                                    result = output[output.index(i) + 1] [1]
                                    dictionary[keyword] = float(result)
                                    break
                                elif ':' in text :
                                    result = output[output.index(i) + 1] [1]
                                    dictionary[keyword] = float(result)
                                    break             
                                elif output[output.index(i) + 1] [1] == ':' :
                                    dictionary[keyword] = float(output[output.index(i) + 2] [1])  
                                    break  
                                elif output[output.index(i) + 1] [1] == '8' :
                                    dictionary[keyword] = float(output[output.index(i) + 2] [1])  
                                    break           
                                else :
                                    result = output[output.index(i) + 1] [1]
                                    dictionary[keyword] = float(result)                               
                        except : 
                            if re.search(keyword, text, re.IGNORECASE):
                                list_  = str(text.split())
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break   
                    except :
                        continue     
    return dictionary                                                   

In [5]:
def extract_img_2(path, keywords):
    img = cv2.imread(path)
    # Convert the image to gray scale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ocr_output = pytesseract.image_to_string(img_gray)
    dictionary = {}
    for line in ocr_output.split('\n'):
        for keyword in keywords:
            try :
                try :
                    if re.search(keyword, line, re.IGNORECASE):
                        if '=' in line :
                            list_  = line.split('=')
                            list_ = str(list_)
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break
                        elif ':' in line :
                            list_  = line.split(':')
                            list_ = str(list_)
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break          
                except : 
                    if re.search(keyword, line, re.IGNORECASE):
                        list_  = str(line.split())
                        result = re.search('\d+\.{0,1}\d*', list_).group()
                        dictionary[keyword] = float(result)
                        break                    
            except :
                continue
                
    return dictionary

In [11]:
extract_img_2('Sample\S4.jpg', ['PTH','Calcium','Phosphorus'])

{'PTH': 8.0, 'Calcium': 6.0, 'Phosphorus': 11.0}

In [19]:

try : 
        try :

                    keywords = ['PTH','Calcium','Phosphorus']
                    extracted_dic = extract_img_1('Sample/S4.jpg', keywords)
                    model = joblib.load('Model/EnsembleModel(RF,DT,GB)(Hyper- Hypo-Anthor-Normal).h5')
                    features  = list(model.feature_names_in_)
                    data_dic = {}
                    data_dic[features[0]] = extracted_dic['PTH']
                    data_dic[features[1]] = extracted_dic['Calcium']
                    data_dic[features[2]] = extracted_dic['Phosphorus']
                    data_dic[features[3]] = np.random.choice([0,1])
                    custom_data = pd.DataFrame(data = [data_dic] )   
                    print('-----'*10)
                    print('Your Input :\n',custom_data)
                    print('-----'*10)       
                    output = model.predict(custom_data)[0] 
                    print('\t Output :',output)

        except :  
                    keywords = ['PTH','Calcium','Phosphorus']
                    extracted_dic = extract_img_2('Sample/S4.jpg', keywords)
                    model = joblib.load('Model/EnsembleModel(RF,DT,GB)(Hyper- Hypo-Anthor-Normal).h5')
                    features  = list(model.feature_names_in_)
                    data_dic = {}
                    data_dic[features[0]] = extracted_dic['PTH']
                    data_dic[features[1]] = extracted_dic['Calcium']
                    data_dic[features[2]] = extracted_dic['Phosphorus']
                    data_dic[features[3]] = np.random.choice([0,1])
                    custom_data = pd.DataFrame(data = [data_dic] )   
                    print('-----'*10)
                    print('Your Input :\n',custom_data)
                    print('-----'*10)       
                    output = model.predict(custom_data)[0] 
                    print('\t Output :',output)
except : 
        print('Your uploaded image can\'t be detected')
        print('-----'*10)       
        model = joblib.load('Model/EnsembleModel(RF,DT,GB)(Hyper- Hypo-Anthor-Normal).h5')
        features  = list(model.feature_names_in_)        
        print('\t Enter Manually the Following Please : ',features[:-1])


--------------------------------------------------
Your Input :
    PTH (pg/ml)  Total_Calcium (mg/dl)  Phosphorus  (mg/dl)  Gender_Male
0          8.0                    6.0                 11.0            0
--------------------------------------------------
	 Output : Hypoparathyroid


C:\Users\intel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\intel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
